In [ ]:
#!/usr/bin/env python
# coding: utf-8

#import the libraries
from __future__ import print_function
import torch
from torch.autograd import Function
import os
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#import torchvision
#from torchvision import datasets, transforms
#import matplotlib.pyplot as plt
import numpy as np
import h5py as h5
from torch.utils import data
#from torchvision import transforms
import itertools
from torch.autograd import Variable
#from sklearn.metrics import confusion_matrix
from glob import glob
import random
import time
import h5py as h5
from osgeo import gdal
import math
import h5py
import matplotlib.pyplot as plt
from PIL import Image

#desktop paths
root_path = "H:/DA_ETH/DRC_BALANCED"


#src_data_dir = root_path + "/BUKAVU_1957/samples/training_buk_57.hdf5"
#src_data_dir = root_path + "/GOMA_1947/samples/training_gom_47.hdf5"
#tgt_data_dir = root_path + "/BUJUMBURA_1957_58_59/samples/training_buj_57.hdf5"


#tgt_data_dir ='H:/DA_ETH/DRC_BALANCED/buj_57_59.hdf5'
#src_data_dir = 'H:/POTSDAM/resample/samples/training_potsdam.hdf5'
#tgt_data_dir = 'H:/ISPRS2DVAIHINGEN/ISPRS_semantic_labeling_Vaihingen/resample/samples/training_vaihingen.hdf5'

'''
root_path = "/work/nmboga/deep_learning"
#src_data_dir = root_path + "/drc_thesis/training_buk_bld_1m.hdf5"
src_data_dir = root_path + "/drc_thesis/tile1_128ps_1_bld.hdf5"    #bukavu 59
tgt_data_dir = root_path +"/drc_thesis/tile1_129patch_size.hdf5"   #goma 47
output_dir = root_path + "/WEIGHTS/"
'''
output_dir = root_path + "/WEIGHTS/"
loss_folder = root_path+'/FIGURES/loss_seg_pot_vaih.hdf5'
#loss_folder = root_path+'/FIGURES/loss_seg_pot_vaih.hdf5'
#src_data_dir = root_path + "/BUKAVU_1959/tile1_128ps_1_bld.hdf5"
#tgt_data_dir = root_path +"/BUJUMBURA_1957_59/tile1_129ps_2.hdf5"
#w8_fname = 'H:/DA_ETH/DRC_BALANCED/WEIGHTS/gom_src_2400.hdf5'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


#N_CLASSES = 5
#IN_CHANNELS = 3

N_CLASSES = 6
IN_CHANNELS = 1

torch.cuda.empty_cache()
seed = 572
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
#
def nulltozero(arr):
    arrcopy = np.copy(arr)
    low_values_flags = arrcopy > 1 #0.7,0  
    arrcopy[low_values_flags]=0
    return arrcopy

def CrossEntropy2d(input, target, weight=None, size_average=True):
    """ 2D version of the cross entropy loss """
    dim = input.dim()
    if dim == 2:
        return F.cross_entropy(input, target, weight, size_average)
    elif dim == 4:
        output = input.view(input.size(0),input.size(1), -1)
        output = torch.transpose(output,1,2).contiguous()
        output = output.view(-1,output.size(2))
        target = target.view(-1)
        return F.cross_entropy(output, target,weight, ignore_index=-1)
    else:
        raise ValueError('Expected 2 or 4 dimensions (got {})'.format(dim))

def fil_ter(arr_x,arr_y):
    '''
    exclude patches with non-building class
    '''
    x_tr = []
    y_tr = []
    for i in range(arr_y.shape[0]):
        if np.max(nulltozero(arr_y[i]))==0:
            pass
        else:
            #np.max(nulltozero(y_train[i]))==1:
            x_tr.append(arr_x[i])
            y_tr.append(arr_y[i])
    x_tr = np.asarray(x_tr)
    y_tr = np.asarray(y_tr)
    return x_tr,y_tr


class MyDataset(data.Dataset):
    
    def __init__(self, archive):
        self.archive = h5.File(archive, 'r')
        self.labels = self.archive['y_train'][:,:,:128,:128]# subtract one, -1,0,1,2,3,4,5 for softmax
        self.data = self.archive['X_train'][:,:,:128,:128]
        
        #filter the data
        self.data,self.labels = fil_ter(self.data,self.labels)
        self.data,self.labels = self.data[:5000,:,:,:],self.labels[:5000,:,:,:]
        
        #self.transform = transform
        
    def __getitem__(self, index):
        return self.data[index], nulltozero(self.labels[index])
        
    def __len__(self):
        return len(self.labels)
    
    def close(self):
        self.archive.close()

class MyDataset(data.Dataset):
    
    def __init__(self, archive):
        self.archive = h5.File(archive, 'r')
        self.labels = self.archive['y_train'][:10000,:,:128,:128]-1# subtract one, -1,0,1,2,3,4,5
        self.data = self.archive['X_train'][:10000,:,:128,:128]
        
        #self.transform = transform
        
    def __getitem__(self, index):
        return self.data[index], self.labels[index]
        
    def __len__(self):
        return len(self.labels)
    
    def close(self):
        self.archive.close()
        
class MyDataset(data.Dataset):
    
    def __init__(self, archive):
        self.archive = h5.File(archive, 'r')
        self.labels = self.archive['y_train'][:10000,:,:128,:128]-1# subtract one, -1,0,1,2,3,4,5
        self.data = self.archive['X_train'][:10000,:,:128,:128]
        
        random.seed(572)
        randomlist = random.sample(range(0, 10000), 100)
        
        self.labels = self.labels[randomlist]
        self.data = self.data[randomlist]
        
        
        #self.transform = transform
        
    def __getitem__(self, index):
        return self.data[index], self.labels[index]
        
    def __len__(self):
        return len(self.labels)
    
    def close(self):
        self.archive.close()
        
def swap_arr(arr):
    """
    #swap axes to that of tf backend (channels last)
    """
    val_1=np.swapaxes(arr,1,3)
    val_2=np.swapaxes(val_1,2,3)
    return val_2
        
class MyDataset(data.Dataset):
    
    def __init__(self, archive):
        self.archive = h5.File(archive, 'r')
        self.labels = self.archive['y_train'][:,:128,:128,:]-1# subtract one, -1,0,1,2,3,4,5
        self.data = self.archive['X_train'][:,:128,:128,:]
        
        #random.seed(572)
        #randomlist = random.sample(range(0, 10000), 100)
        
        #self.labels = self.labels[randomlist]
        #self.data = self.data[randomlist]
        print(self.labels.shape)
        
        self.labels = swap_arr(self.labels)
        #print(self.labels.shape)
        self.data =swap_arr(self.data)
        
        
        #self.transform = transform
        
    def __getitem__(self, index):
        return self.data[index], self.labels[index]
        
    def __len__(self):
        return len(self.labels)
    
    def close(self):
        self.archive.close()


class MyDataset_BK(data.Dataset):
    #bukavu
    def __init__(self, archive,slice_1 = 1600, slice_2 = 400):
        self.archive = h5.File(archive, 'r')
        self.labels = self.archive['y_train'][:,:,:128,:128]-1# subtract one, -1,0,1,2,3,4,5
        self.data = self.archive['X_train'][:,:,:128,:128]
        
        random.seed(572)
        list_1 = random.sample(range(0, 5195), slice_1)
        list_2 = random.sample(range(5196, 7180),slice_2)
        list_3 = random.sample(range(7181, 10000), slice_2)

        list_4 = list_1+list_2+list_3 
        self.labels = self.labels[list_4]
        self.data = self.data[list_4]
        print(np.unique(self.labels,return_counts=True))
        
        
        #self.transform = transform
        
    def __getitem__(self, index):
        return self.data[index], self.labels[index]
        
    def __len__(self):
        return len(self.labels)
    
    def close(self):
        self.archive.close()
        
class MyDataset_GM(data.Dataset):
    #goma 47
    def __init__(self, archive,slice_1=1600,slice_2=400):
        self.archive = h5.File(archive, 'r')
        self.labels = self.archive['y_train'][:,:,:128,:128]-1# subtract one, -1,0,1,2,3,4,5
        self.data = self.archive['X_train'][:,:,:128,:128]
        
        random.seed(572)
        list_1 = random.sample(range(0, 3773), slice_1)
        list_2 = random.sample(range(3774, 8567),slice_2)
        list_3 = random.sample(range(8567, 10000), slice_2)

        list_4 = list_1+list_2+list_3 
        self.labels = self.labels[list_4]
        self.data = self.data[list_4]
        print(np.unique(self.labels,return_counts=True))
        
        
        #self.transform = transform
        
    def __getitem__(self, index):
        return self.data[index], self.labels[index]
        
    def __len__(self):
        return len(self.labels)
    
    def close(self):
        self.archive.close()
class MyDataset_BK(data.Dataset):
    #bukavu
    def __init__(self, archive,slice_1 = 1600, slice_2 = 400):
        self.archive = h5.File(archive, 'r')
        self.labels = self.archive['y_train'][:,:,:128,:128]-1# subtract one, -1,0,1,2,3,4,5
        self.data = self.archive['X_train'][:,:,:128,:128]
        
        random.seed(572)
        list_1 = random.sample(range(0, 5195), slice_1)
        list_2 = random.sample(range(5196, 7180),slice_2)
        list_3 = random.sample(range(7181, 10000), slice_2)

        list_4 = list_1+list_2+list_3 
        self.labels = self.labels[list_4]
        self.data = self.data[list_4]
        print(np.unique(self.labels,return_counts=True))
        
        
        #self.transform = transform
        
    def __getitem__(self, index):
        return self.data[index], self.labels[index]
        
    def __len__(self):
        return len(self.labels)
    
    def close(self):
        self.archive.close()     
        
# NO RANDOM SAMPLING      
class MyDataset_GM(data.Dataset):
    #goma 47
    def __init__(self, archive,slice_1=1600,slice_2=400):
        self.archive = h5.File(archive, 'r')
        self.labels = self.archive['y_train'][:,:,:128,:128]-1# subtract one, -1,0,1,2,3,4,5
        self.data = self.archive['X_train'][:,:,:128,:128]
        
        random.seed(572)
        #list_1 = random.sample(range(0, 3773), 300)
        #list_2 = random.sample(range(3774, 8567),75)
        #list_3 = random.sample(range(8567, 10000), 75)
        
        l_1 = list (range(slice_1))
        l_2 = list (range(slice_2))
        l_3 = list (range(slice_2))
        
        #list_2 = list (range(3))
        list_1 = [x + 0 for x in l_1] 
        list_2 = [x + 3774 for x in l_2] 
        list_3 = [x + 8567 for x in l_3] 
        

        list_4 = list_1+list_2+list_3 
        self.labels = self.labels[list_4]
        self.data = self.data[list_4]
        print(np.unique(self.labels,return_counts=True))
        
        
        #self.transform = transform
        
    def __getitem__(self, index):
        return self.data[index], self.labels[index]
        
    def __len__(self):
        return len(self.labels)
    
    def close(self):
        self.archive.close()
    
class MyDataset_BK(data.Dataset):
    #bukavu 59
    def __init__(self, archive,slice_1=1600,slice_2=400):
        self.archive = h5.File(archive, 'r')
        self.labels = self.archive['y_train'][:,:,:128,:128]-1# subtract one, -1,0,1,2,3,4,5
        self.data = self.archive['X_train'][:,:,:128,:128]
        
        random.seed(572)
        #list_1 = random.sample(range(0, 3773), 300)
        #list_2 = random.sample(range(3774, 8567),75)
        #list_3 = random.sample(range(8567, 10000), 75)
        
        l_1 = list (range(slice_1))
        l_2 = list (range(slice_2))
        l_3 = list (range(slice_2))
        
        #list_2 = list (range(3))
        list_1 = [x + 0 for x in l_1] 
        list_2 = [x + 5195 for x in l_2] 
        list_3 = [x + 7180 for x in l_3] 
        

        list_4 = list_1+list_2+list_3 
        self.labels = self.labels[list_4]
        self.data = self.data[list_4]
        print(np.unique(self.labels,return_counts=True))
        
        
        #self.transform = transform
        
    def __getitem__(self, index):
        return self.data[index], self.labels[index]
        
    def __len__(self):
        return len(self.labels)
    
    def close(self):


# random sampling with s        
        
class MyDataset_GM(data.Dataset):
    #goma 47
    def __init__(self, archive,slice_3=500,seed_value=10):
        self.archive = h5.File(archive, 'r')
        self.labels = self.archive['y_train'][:,:,:128,:128]-1# subtract one, -1,0,1,2,3,4,5
        self.data = self.archive['X_train'][:,:,:128,:128]
        
        random.seed(572)
        list_1 = random.sample(range(0, 3773), 2000)
        list_2 = random.sample(range(3774, 8567),500)
        list_3 = random.sample(range(8567, 10000), 500)

        list_4 = list_1+list_2+list_3 
        self.labels = self.labels[list_4]
        self.data = self.data[list_4]
        print(np.unique(self.labels,return_counts=True))
        
        #introduce the sequential seed values
        random.seed(seed_value)
        list_5 = random.sample(range(0, 3000), slice_3)
        self.labels = self.labels[list_5]
        self.data = self.data[list_5]
        print(np.unique(self.labels,return_counts=True))
        
        
        #self.transform = transform
        
    def __getitem__(self, index):
        return self.data[index], self.labels[index]
        
    def __len__(self):
        return len(self.labels)
    
    def close(self):
        self.archive.close()
        
class MyDataset_BK(data.Dataset):
    #bukavu
    def __init__(self, archive, slice_3 = 500, seed_value=10):
        self.archive = h5.File(archive, 'r')
        self.labels = self.archive['y_train'][:,:,:128,:128]-1# subtract one, -1,0,1,2,3,4,5
        self.data = self.archive['X_train'][:,:,:128,:128]
        
        random.seed(572)
        list_1 = random.sample(range(0, 5195), 2000)
        list_2 = random.sample(range(5196, 7180),500)
        list_3 = random.sample(range(7181, 10000), 500)

        list_4 = list_1+list_2+list_3 
        self.labels = self.labels[list_4]
        self.data = self.data[list_4]
        print(np.unique(self.labels,return_counts=True))
        
        #introduce the sequential seed values
        random.seed(seed_value)
        list_5 = random.sample(range(0, 3000), slice_3)
        self.labels = self.labels[list_5]
        self.data = self.data[list_5]
        print(np.unique(self.labels,return_counts=True))
        
        
        #self.transform = transform
        
    def __getitem__(self, index):
        return self.data[index], self.labels[index]
        
    def __len__(self):
        return len(self.labels)
    
    def close(self):
        self.archive.close() 

'''
class MyDataset(data.Dataset):
    #bujuumbura
    def __init__(self, archive):
        self.archive = h5.File(archive, 'r')
        self.labels = self.archive['y_train'][:,:,:128,:128]-1# subtract one, -1,0,1,2,3,4,5
        self.data = self.archive['X_train'][:,:,:128,:128]
        
        random.seed(572)
        list_1 = random.sample(range(0, 6321), 2000)
        list_2 = random.sample(range(6322, 7072), 500)
        list_3 = random.sample(range(7073, 10000), 500)

        list_4 = list_1+list_2+list_3 
        self.labels = self.labels[list_4]
        self.data = self.data[list_4]
        print(np.unique(self.labels,return_counts=True))
        
        
        #self.transform = transform
        
    def __getitem__(self, index):
        return self.data[index], self.labels[index]
        
    def __len__(self):
        return len(self.labels)
    
    def close(self):
        self.archive.close()
'''
        
class ReverseLayerF(Function):

    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha
        return output, None
class EncDec(nn.Module):
    def __init__(self, in_channels=IN_CHANNELS, out_channels=N_CLASSES):
        super(EncDec,self).__init__()
        self.pool = nn.MaxPool2d(2)
        #self.bn_3 = nn.BatchNorm2d(1024)
        
        self.conv1_1 = nn.Conv2d(in_channels, 64, kernel_size=3, padding = 1)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, padding = 1)
        self.bn_1 = nn.BatchNorm2d(64)
        
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, padding = 1)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding = 1)
        self.bn_2 = nn.BatchNorm2d(128)
        
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding = 1)
        self.conv3_2= nn.Conv2d(256, 256, kernel_size=3, padding = 1)
        self.bn_3 = nn.BatchNorm2d(256)
        
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, padding = 1)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, padding = 1)
        self.bn_4 = nn.BatchNorm2d(512)
        
        self.conv5_1 = nn.Conv2d(512, 1024, kernel_size=3, padding = 1)
        self.conv5_2 = nn.Conv2d(1024, 1024, kernel_size=3, padding = 1)
        self.bn_5 = nn.BatchNorm2d(1024)
    
        #conv transpose layers
        self.conv1_t = nn.ConvTranspose2d(1024, 512, stride=2, kernel_size=3, padding = 1,output_padding =1)
        self.conv2_t = nn.ConvTranspose2d(512, 256, stride=2, kernel_size=3, padding = 1,output_padding =1)
        self.conv3_t = nn.ConvTranspose2d(256, 128, stride=2, kernel_size=3, padding = 1,output_padding =1)
        self.conv4_t = nn.ConvTranspose2d(128, 64, stride=2, kernel_size=3, padding = 1,output_padding =1)
        
        #upsampling layers
        
        self.conv4_1_U = nn.Conv2d(512, 512, kernel_size=3, padding = 1)
        self.conv4_2_U = nn.Conv2d(512, 512, kernel_size=3, padding = 1)
        self.bn_4_U = nn.BatchNorm2d(512)
        
        self.conv3_1_U = nn.Conv2d(256, 256, kernel_size=3, padding = 1)
        self.conv3_2_U = nn.Conv2d(256, 256, kernel_size=3, padding = 1)
        self.bn_3_U = nn.BatchNorm2d(256)
        
        self.conv2_1_U = nn.Conv2d(128, 128, kernel_size=3, padding = 1)
        self.conv2_2_U = nn.Conv2d(128, 128, kernel_size=3, padding = 1)
        self.bn_2_U = nn.BatchNorm2d(128)
        
        self.conv1_1_U = nn.Conv2d(64, 64, kernel_size=3, padding = 1)
        self.conv1_2_U = nn.Conv2d(64, 64, kernel_size=3, padding = 1)
        self.bn_1_U = nn.BatchNorm2d(64)
        
        # final output layer
        #self.conv_sm = nn.Conv2d(64, out_channels, kernel_size=1, padding = 0)
        self.conv_sm = nn.Conv2d(64, out_channels, kernel_size=3, padding=1)
        
        
        # discriminator
        self.discriminator = nn.Sequential(
            nn.Linear(1024*8*8, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(inplace=True),
            nn.Linear(100, 2)
        )
        
    def forward(self,x,alpha):
        conv1_d = F.relu(self.conv1_1(x))
        conv1_d = self.bn_1(F.relu(self.conv1_2(conv1_d)))
        x = self.pool(conv1_d)
        #conv1_d = self.pool()
        
        conv2_d = F.relu(self.conv2_1(x))
        conv2_d = self.bn_2(F.relu(self.conv2_2(conv2_d)))
        x = self.pool(conv2_d)
        
        conv3_d = F.relu(self.conv3_1(x))
        conv3_d = self.bn_3(F.relu(self.conv3_2(conv3_d)))
        x = self.pool(conv3_d)
        
        conv4_d = F.relu(self.conv4_1(x))
        conv4_d = self.bn_4(F.relu(self.conv4_2(conv4_d)))
        x = self.pool(conv4_d)
        
        conv5_d = F.relu(self.conv5_1(x))
        conv5_d = self.bn_5(F.relu(self.conv5_2(conv5_d)))
        
        # x is the output to the domain and scene classfier
        
        x = F.relu(self.conv1_t(conv5_d))
        #merge1 = torch.cat((x,conv4_d),1)
        x = F.relu(self.conv4_1_U(x))
        x = self.bn_4_U(F.relu(self.conv4_2_U(x)))
        
        x = F.relu(self.conv2_t(x))
        #merge2 = torch.cat((x,conv3_d),1)
        x = F.relu(self.conv3_1_U(x))
        x = self.bn_3_U(F.relu(self.conv3_2_U(x)))
        
        x = F.relu(self.conv3_t(x))
        #merge3 = torch.cat((x,conv2_d),1)
        x = F.relu(self.conv2_1_U(x))
        x = self.bn_2_U(F.relu(self.conv2_2_U(x)))
        
        x = F.relu(self.conv4_t(x))
        #merge4 = torch.cat((x,conv1_d),1)
        x = F.relu(self.conv1_1_U(x))
        x = self.bn_1_U(F.relu(self.conv1_2_U(x)))
        #print(x.shape)
        
        #classification layer
        class_output = self.conv_sm(x)
        #print(class_output.shape)
        
        #x_f output of feature extractio
        x_f = conv5_d.view(-1,1024*8*8)
        reverse_feature = ReverseLayerF.apply(x_f,alpha) # do not apply the reversed gradient
        domain_output = self.discriminator(reverse_feature)
        #domain_output = self.discriminator(x_f)
        return class_output, domain_output


class UNET(nn.Module):
    def __init__(self, in_channels=IN_CHANNELS, out_channels=N_CLASSES):
        super(UNET,self).__init__()
        self.pool = nn.MaxPool2d(2)
        #self.bn_3 = nn.BatchNorm2d(1024)
        
        self.conv1_1 = nn.Conv2d(in_channels, 64, kernel_size=3, padding = 1)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, padding = 1)
        self.bn_1 = nn.BatchNorm2d(64)
        
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, padding = 1)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding = 1)
        self.bn_2 = nn.BatchNorm2d(128)
        
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding = 1)
        self.conv3_2= nn.Conv2d(256, 256, kernel_size=3, padding = 1)
        self.bn_3 = nn.BatchNorm2d(256)
        
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, padding = 1)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, padding = 1)
        self.bn_4 = nn.BatchNorm2d(512)
        
        self.conv5_1 = nn.Conv2d(512, 1024, kernel_size=3, padding = 1)
        self.conv5_2 = nn.Conv2d(1024, 1024, kernel_size=3, padding = 1)
        self.bn_5 = nn.BatchNorm2d(1024)
    
        #conv transpose layers
        self.conv1_t = nn.ConvTranspose2d(1024, 512, stride=2, kernel_size=3, padding = 1,output_padding =1)
        self.conv2_t = nn.ConvTranspose2d(512, 256, stride=2, kernel_size=3, padding = 1,output_padding =1)
        self.conv3_t = nn.ConvTranspose2d(256, 128, stride=2, kernel_size=3, padding = 1,output_padding =1)
        self.conv4_t = nn.ConvTranspose2d(128, 64, stride=2, kernel_size=3, padding = 1,output_padding =1)
        
        #upsampling layers
        
        self.conv4_1_U = nn.Conv2d(1024, 512, kernel_size=3, padding = 1)
        self.conv4_2_U = nn.Conv2d(512, 512, kernel_size=3, padding = 1)
        self.bn_4_U = nn.BatchNorm2d(512)
        
        self.conv3_1_U = nn.Conv2d(512, 256, kernel_size=3, padding = 1)
        self.conv3_2_U = nn.Conv2d(256, 256, kernel_size=3, padding = 1)
        self.bn_3_U = nn.BatchNorm2d(256)
        
        self.conv2_1_U = nn.Conv2d(256, 128, kernel_size=3, padding = 1)
        self.conv2_2_U = nn.Conv2d(128, 128, kernel_size=3, padding = 1)
        self.bn_2_U = nn.BatchNorm2d(128)
        
        self.conv1_1_U = nn.Conv2d(128, 64, kernel_size=3, padding = 1)
        self.conv1_2_U = nn.Conv2d(64, 64, kernel_size=3, padding = 1)
        self.bn_1_U = nn.BatchNorm2d(64)
        
        # final output layer
        #self.conv_sm = nn.Conv2d(64, out_channels, kernel_size=1, padding = 0)
        self.conv_sm = nn.Conv2d(64, out_channels, kernel_size=3, padding=1)
        
        
        # discriminator
        self.discriminator = nn.Sequential(
            nn.Linear(1024*8*8, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(inplace=True),
            nn.Linear(100, 2)
        )
        
    def forward(self,x,alpha):
        conv1_d = F.relu(self.conv1_1(x))
        conv1_d = self.bn_1(F.relu(self.conv1_2(conv1_d)))
        x = self.pool(conv1_d)
        #conv1_d = self.pool()
        
        conv2_d = F.relu(self.conv2_1(x))
        conv2_d = self.bn_2(F.relu(self.conv2_2(conv2_d)))
        x = self.pool(conv2_d)
        
        conv3_d = F.relu(self.conv3_1(x))
        conv3_d = self.bn_3(F.relu(self.conv3_2(conv3_d)))
        x = self.pool(conv3_d)
        
        conv4_d = F.relu(self.conv4_1(x))
        conv4_d = self.bn_4(F.relu(self.conv4_2(conv4_d)))
        x = self.pool(conv4_d)
        
        conv5_d = F.relu(self.conv5_1(x))
        conv5_d = self.bn_5(F.relu(self.conv5_2(conv5_d)))
        
        # x is the output to the domain and scene classfier
        
        x = F.relu(self.conv1_t(conv5_d))
        merge1 = torch.cat((x,conv4_d),1)
        x = F.relu(self.conv4_1_U(merge1))
        x = self.bn_4_U(F.relu(self.conv4_2_U(x)))
        
        x = F.relu(self.conv2_t(x))
        merge2 = torch.cat((x,conv3_d),1)
        x = F.relu(self.conv3_1_U(merge2))
        x = self.bn_3_U(F.relu(self.conv3_2_U(x)))
        
        x = F.relu(self.conv3_t(x))
        merge3 = torch.cat((x,conv2_d),1)
        x = F.relu(self.conv2_1_U(merge3))
        x = self.bn_2_U(F.relu(self.conv2_2_U(x)))
        
        x = F.relu(self.conv4_t(x))
        merge4 = torch.cat((x,conv1_d),1)
        x = F.relu(self.conv1_1_U(merge4))
        x = self.bn_1_U(F.relu(self.conv1_2_U(x)))
        #print(x.shape)
        
        #classification layer
        class_output = self.conv_sm(x)
        #print(class_output.shape)
        
        #x_f output of feature extractio
        x_f = conv5_d.view(-1,1024*8*8)
        reverse_feature = ReverseLayerF.apply(x_f,alpha) # do not apply the reversed gradient
        domain_output = self.discriminator(reverse_feature)
        #domain_output = self.discriminator(x_f)
        return class_output, domain_output

def DeepCoral(source, target):
    d = source.data.shape[1]
    xm = torch.mean(source, 1, keepdim=True)
    xc = torch.matmul(torch.transpose(xm, 0, 1), xm)  # source covariance
    xmt = torch.mean(target, 1, keepdim=True)
    xct = torch.matmul(torch.transpose(xmt, 0, 1), xmt)   # target covariance
    loss = torch.mean(torch.mul((xc - xct), (xc - xct)))   # frobenius norm between source and target
    return loss/(4*d*d)

class UNET(nn.Module):
    def __init__(self, in_channels=IN_CHANNELS, out_channels=N_CLASSES):
        super(UNET,self).__init__()
        self.pool = nn.MaxPool2d(2)
        #self.bn_3 = nn.BatchNorm2d(1024)
        
        self.conv1_1 = nn.Conv2d(in_channels, 64, kernel_size=3, padding = 1)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, padding = 1)
        self.bn_1 = nn.BatchNorm2d(64)
        
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, padding = 1)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding = 1)
        self.bn_2 = nn.BatchNorm2d(128)
        
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding = 1)
        self.conv3_2= nn.Conv2d(256, 256, kernel_size=3, padding = 1)
        self.bn_3 = nn.BatchNorm2d(256)
        
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, padding = 1)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, padding = 1)
        self.bn_4 = nn.BatchNorm2d(512)
        
        self.conv5_1 = nn.Conv2d(512, 1024, kernel_size=3, padding = 1)
        self.conv5_2 = nn.Conv2d(1024, 1024, kernel_size=3, padding = 1)
        self.bn_5 = nn.BatchNorm2d(1024)
    
        #conv transpose layers
        self.conv1_t = nn.ConvTranspose2d(1024, 512, stride=2, kernel_size=3, padding = 1,output_padding =1)
        self.conv2_t = nn.ConvTranspose2d(512, 256, stride=2, kernel_size=3, padding = 1,output_padding =1)
        self.conv3_t = nn.ConvTranspose2d(256, 128, stride=2, kernel_size=3, padding = 1,output_padding =1)
        self.conv4_t = nn.ConvTranspose2d(128, 64, stride=2, kernel_size=3, padding = 1,output_padding =1)
        
        #upsampling layers
        
        self.conv4_1_U = nn.Conv2d(1024, 512, kernel_size=3, padding = 1)
        self.conv4_2_U = nn.Conv2d(512, 512, kernel_size=3, padding = 1)
        self.bn_4_U = nn.BatchNorm2d(512)
        
        self.conv3_1_U = nn.Conv2d(512, 256, kernel_size=3, padding = 1)
        self.conv3_2_U = nn.Conv2d(256, 256, kernel_size=3, padding = 1)
        self.bn_3_U = nn.BatchNorm2d(256)
        
        self.conv2_1_U = nn.Conv2d(256, 128, kernel_size=3, padding = 1)
        self.conv2_2_U = nn.Conv2d(128, 128, kernel_size=3, padding = 1)
        self.bn_2_U = nn.BatchNorm2d(128)
        
        self.conv1_1_U = nn.Conv2d(128, 64, kernel_size=3, padding = 1)
        self.conv1_2_U = nn.Conv2d(64, 64, kernel_size=3, padding = 1)
        self.bn_1_U = nn.BatchNorm2d(64)
        
        # final output layer
        #self.conv_sm = nn.Conv2d(64, out_channels, kernel_size=1, padding = 0)
        self.conv_sm = nn.Conv2d(64, out_channels, kernel_size=3, padding=1)
        
        
        # discriminator
        self.discriminator = nn.Sequential(
            nn.Linear(1024*8*8, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(inplace=True),
            nn.Linear(100, 2)
        )
        
    def forward(self,x):
        conv1_d = F.relu(self.conv1_1(x))
        conv1_d = self.bn_1(F.relu(self.conv1_2(conv1_d)))
        x = self.pool(conv1_d)
        #conv1_d = self.pool()
        
        conv2_d = F.relu(self.conv2_1(x))
        conv2_d = self.bn_2(F.relu(self.conv2_2(conv2_d)))
        x = self.pool(conv2_d)
        
        conv3_d = F.relu(self.conv3_1(x))
        conv3_d = self.bn_3(F.relu(self.conv3_2(conv3_d)))
        x = self.pool(conv3_d)
        
        conv4_d = F.relu(self.conv4_1(x))
        conv4_d = self.bn_4(F.relu(self.conv4_2(conv4_d)))
        x = self.pool(conv4_d)
        
        conv5_d = F.relu(self.conv5_1(x))
        conv5_d = self.bn_5(F.relu(self.conv5_2(conv5_d)))
        
        # x is the output to the domain and scene classfier
        
        x = F.relu(self.conv1_t(conv5_d))
        merge1 = torch.cat((x,conv4_d),1)
        x = F.relu(self.conv4_1_U(merge1))
        x = self.bn_4_U(F.relu(self.conv4_2_U(x)))
        
        x = F.relu(self.conv2_t(x))
        merge2 = torch.cat((x,conv3_d),1)
        x = F.relu(self.conv3_1_U(merge2))
        x = self.bn_3_U(F.relu(self.conv3_2_U(x)))
        
        x = F.relu(self.conv3_t(x))
        merge3 = torch.cat((x,conv2_d),1)
        x = F.relu(self.conv2_1_U(merge3))
        x = self.bn_2_U(F.relu(self.conv2_2_U(x)))
        
        x = F.relu(self.conv4_t(x))
        merge4 = torch.cat((x,conv1_d),1)
        x = F.relu(self.conv1_1_U(merge4))
        x = self.bn_1_U(F.relu(self.conv1_2_U(x)))
        #print(x.shape)
        
        #classification layer
        class_output = self.conv_sm(x)
        #print(class_output.shape)
        
        #x_f output of feature extractio
        bottleneck_feature = conv5_d.view(-1,1024*8*8)
        #reverse_feature = ReverseLayerF.apply(x_f,alpha) # do not apply the reversed gradient
        #domain_output = self.discriminator(reverse_feature)
        #domain_output = self.discriminator(x_f)
        #return class_output, domain_output
        return class_output,bottleneck_feature

#sgm = torch.nn.Sigmoid()
#x = sgm(self.conv_sm(x))
#criterion_1 = torch.nn.BCEWithLogitsLoss()

#weights = [5.55,2.26,2.01,1,1.8,2.0]
#weights = [3.0,2.26,2.01,1,1.8,2.0]
#weights = [11,4,1,1,1,1]
weights = [1,1,1,1,1,1]

class_weights = torch.FloatTensor(weights).cuda()
#self.criterion = nn.CrossEntropyLoss(weight=class_weights)

##define training schedule for da

def train(net, optimizer, epochs, src_data_loader,tgt_data_loader,device,scheduler=None, weights=None, save_epoch = 5):
    #weights = weights.cuda() # weights for class balancing
    #criterion = nn.NLLLoss(weight = weights)
    #criterion_1 = torch.nn.BCEWithLogitsLoss() #discriminator
    criterion_1 = nn.CrossEntropyLoss() #discriminator
    
    criterion_2 = nn.CrossEntropyLoss(weight = None,ignore_index=-1) #classifier
    val_loss_history = []
    
    src_cl_loss_history = []
    #src_dl_loss_history = []
    #tgt_dl_loss_history = []
    tra_acc_history = []
    coral_loss_history = []
    #dom_acc_history=[]
    m=[]
    
    iter_ = 0
    best_acc =0.0
    for epoch in range(1, epochs + 1):
        print("epoch number %d"%epoch)
        print('-'*10)
        if scheduler is not None:
            scheduler.step()
        net.train()
        
        len_dataloader = min(len(src_data_loader), len(tgt_data_loader))
        data_zip = enumerate(zip(src_data_loader,tgt_data_loader))
        running_loss = 0.0
        run_src_cl = 0.0
        run_src_dl = 0.0
        #run_tgt_dl = 0.0
        run_acc = 0.0
        d_acc = 0.0
        run_coral = 0.0
        
        dom_preds = []
        
        for batch_idx, ((images_src,class_src),(images_tgt,_)) in data_zip:
            p = float(batch_idx + epoch * len_dataloader) /\
            epochs / len_dataloader
            #alpha = 2. / (1. + np.exp(-10 * p)) - 1
            #alpha =1
            #depreciate the lr
            #lr = 0.01 / (1. + 10 * p)**0.75
            #optimizer.lr = lr
            
            size_src = len(images_src)
            size_tgt = len(images_tgt)
            #label_src = torch.zeros(size_src,1).long().to(device) # source 0 #change to long for CEL
            #label_tgt = torch.ones(size_tgt,1).long().to(device) # target 1
            
            # for CEL no need ofr extra dim
            label_src = torch.zeros(size_src).long().to(device) # source 0 #change to long for CEL
            label_tgt = torch.ones(size_tgt).long().to(device) # target 1
            
            #labels = torch.cat((torch.zeros(size_src,1).float(),torch.ones(size_tgt,1).float()),0).to(device)
            
            #make images variable
            class_src = class_src.squeeze(1).to(device) #cross entropy
            #class_src = class_src.to(device)
            
            images_src = images_src.to(device)
            images_tgt = images_tgt.to(device)
            #images = torch.cat((images_src,images_tgt),0).to(device)
            
            #print(torch.max(class_src),torch.min(class_src))
        
            #data, target = Variable(data.cuda()), Variable(target.cuda())
            #zero grad for the optimizer
            
            optimizer.zero_grad()
            #train on the images
            #images = torch.cat((images_src,images_tgt),0).to(device)
            #class_output, domain_output = net(x=images, alpha = alpha)
            #img_loss = criterion_1(domain_output,labels)

            #train on source domain
            src_class_output, src_domain_output = net(x=images_src.float().to(device))
            src_loss_class = criterion_2(src_class_output,class_src.long())
            #print(label_src)
            #print(src_domain_output)
            
            
            #src_loss_domain = criterion_1(src_domain_output, label_src)
            
            #train on target domain
            _, tgt_domain_output =net(x=images_tgt.float())
            #tgt_loss_domain = criterion_1(tgt_domain_output, label_tgt)
            
            # COMPUTE THE  CORAL LOSS
            
            lamb_da = epoch/epochs
            
            coral_loss = DeepCoral(src_domain_output,tgt_domain_output)
            #loss = src_loss_class + lamb_da*coral_loss

            loss = src_loss_class
            
            loss.backward()
            optimizer.step()
            
            #pred the domain class
            #tsm = torch.nn.Sigmoid()
            #src_dom_class = tsm(src_domain_output)
            #src_dom_class = src_dom_class.data.cpu().numpy()
            #domain accuracy
            #dom_pred = torch.argmax(src_domain_output,1)
            #dom_acc = (dom_pred == label_src).float().mean()
            #print(dom_acc)
            #d_acc+=dom_acc.item()*images_src.size(0)
            
            
            #get the mean
            #dom_src_lab = torch.zeros(size_src,1).to(device)
            #s = (src_dom_class<0.5)
            #d = (dom_src_lab==0)
            #t = (s==d).float().mean()
            #print(d)
            #dom_preds.append(src_dom_class)
            #print('src_dom class',src_dom_class)

            #print(src_dom_class)
            
            pred = torch.argmax(src_class_output, 1)
            acc = (pred == class_src).float().mean()
            #print(acc)
            run_acc+=acc.item()*images_src.size(0)
            
            #saving the loss
            running_loss +=loss.item()*images_src.size(0)
            #running_loss +=loss.item()*images.size(0)
            
            #the individual losses
            run_src_cl += src_loss_class.item()*images_src.size(0)
            run_coral += coral_loss.item()*images_src.size(0)
            #run_src_dl += src_loss_domain.item()*images_src.size(0)
            #run_tgt_dl += tgt_loss_domain.item()*images_src.size(0)
            #print(images_src.size(0))
            
            iter_+=1
        
        epoch_loss = running_loss/ (len(src_data_loader.dataset)+len(src_data_loader.dataset))
        val_loss_history.append(epoch_loss)
        
        tra_acc_history.append(run_acc/len(src_data_loader.dataset))
        #dom_acc_history.append(d_acc/len(src_data_loader.dataset))
       
        #save the individual losses to a list
        epoch_loss = run_src_cl/ len(src_data_loader.dataset)
        src_cl_loss_history.append(epoch_loss)
        
        # coral loss
        
        epoch_loss = run_coral/ len(src_data_loader.dataset)
        coral_loss_history.append(epoch_loss)
        
        
        #epoch_loss = run_src_dl/ len(src_data_loader.dataset)
        #src_dl_loss_history.append(epoch_loss)
        
        #epoch_loss = run_tgt_dl/ len(src_data_loader.dataset)
        #tgt_dl_loss_history.append(epoch_loss)
        #print(len(src_data_loader.dataset))
        
        #saving a checkpoint
        #acc1 = run_acc/len(src_data_loader.dataset)
        #is_best = acc1>best_acc
        #best_acc = max(acc1,best_acc)
        #print(best_acc)
        #if is_best:
            #torch.save(net.state_dict(), output_dir+str(epoch)+"_buj_100.hdf5")
            #torch.save(net.state_dict(), w8_fname)
            #print("saved epoch",epoch)
    torch.save(net.state_dict(), w8_fname)
    #return val_loss_history,tra_acc_history,dom_acc_history,src_cl_loss_history,src_dl_loss_history,tgt_dl_loss_history
    return val_loss_history, tra_acc_history, coral_loss_history, src_cl_loss_history

##########
#FUNCTIONS FOR TESTING STAGE
##########

#testing for one class
## functions for testing 
import itertools
import subprocess, glob
from random import seed
from sklearn import metrics
from sklearn.metrics import confusion_matrix
seed(1234)


BATCH_SIZE=8
WINDOW_SIZE=[256,256]
patch_size=256
STRIDE=228
nc=6
gmax=63450
#gmax=45000
gmin=0

# CREATE DIRECTORIES
def make_dir_paths(mypath):
    if not os.path.isdir(mypath):
        os.makedirs(mypath)
        print("created path")

## REMOVE NULLS
def nulltozero(arr):
    arrcopy = np.copy(arr)
    low_values_flags = arrcopy < 0 #0.7,0  
    arrcopy[low_values_flags]=0
    return arrcopy

#LOAD IMAGES AND CONVERT TO ARRAYS
def img_to_array(*images):
    """Convert an image or list of images to numpy arrays.

    Keyword arguments:
    *images -- list containing the images to be converted
    """
    imgarrays = []
    i = 0
    for img in images:
        arr = gtiff_to_array(img)
        imgarrays.append(arr)
    return imgarrays


def gtiff_to_array(imgfname):                                      
    """Transform a geotiff to numpy array.

    Keyword arguments:
    imgfnames -- filename of image to convert
    """
    ds = gdal.Open(imgfname)
    for band in range(ds.RasterCount):
        band += 1
        if band == 1:
            arr = np.array(ds.GetRasterBand(band).ReadAsArray())
            arr = np.expand_dims(arr, axis=2)
        else:
            concat = np.array(ds.GetRasterBand(band).ReadAsArray())
            concat = np.expand_dims(concat, axis=2)
            arr = np.concatenate((arr,
                                  concat),
                                 axis=2)
    return arr

def reclassgts2(gtsarray):
    """Reclassify ground truth dataset array to single class numbers.

    Keyword arguments:
    gtsarray -- the ground truth dataset array
    """
    reclassarray = np.zeros(shape=(gtsarray.shape[0], gtsarray.shape[1]),
                            dtype=np.uint8)
    cnum = 1
    mask = np.logical_or(np.logical_or(gtsarray[:,:,0]==1,gtsarray[:,:,0]==3),gtsarray[:,:,0]==4)
    reclassarray[mask]=cnum

    cnum2 = 2
    mask = np.logical_or(gtsarray[:,:,0]==2,gtsarray[:,:,0]==2)
    reclassarray[mask] = cnum2
    return reclassarray

#create idxarray for the creation of the map
def sample_idx(arr):
    """Randomly sample an array stratified based on frequency.

    Keyword arguments:
    arr -- the array being sampled
    cratios -- the representative fractions of each classes
    n -- total number of samples (default 1000)
    """
    i = 0
    arr_copy = np.copy(arr)
    idxarray = np.zeros(shape=(0, 2), dtype=np.int16)
    nc = np.array(np.where(arr_copy >0)).T.shape[0]
    arr_flat = arr_copy.flatten()
    n = arr_flat.shape[0]
    randidx = np.asarray(range(n),dtype= np.int32)
    #randidx=randidx.astype(np.int32)
    idxarray = np.array(np.where(arr_copy >= 0)).T[randidx, :]
    #sampleidx += csamples

    del arr_copy
    return idxarray

def write_geotiff(fname, data, geo_transform, projection):
    """Create a GeoTIFF file with the given data."""
    driver = gdal.GetDriverByName('GTiff')
    rows, cols = data.shape
    dataset = driver.Create(fname, cols, rows, 1, gdal.GDT_Byte)
    dataset.SetGeoTransform(geo_transform)
    dataset.SetProjection(projection)
    band = dataset.GetRasterBand(1)
    band.WriteArray(data)
    dataset = None  # Close the file

def swap_axes(arr):
    return np.expand_dims(arr, axis = 0)

#REMOVE NAN
def nulltozero_g(arr):
    arrcopy = np.copy(arr)
    low_values_flags = (np.isnan(arr))#0.7,0
    arrcopy[low_values_flags]=0
    return arrcopy

def extract_geometry(path_dir):
    raster_dataset = gdal.Open(path_dir, gdal.GA_ReadOnly)
    geo_transform = raster_dataset.GetGeoTransform()

    return geo_transform

# Utils adapting the prediction scheme in https://github.com/nshaud/DeepNetsForEO/blob/master/SegNet_PyTorch_v2.ipynb
#https://machinelearningmastery.com/reproducible-results-neural-networks-keras/


def get_random_pos(img, window_shape):
    """ Extract of 2D random patch of shape window_shape in the image """
    w, h = window_shape
    W, H = img.shape[-2:]
    x1 = random.randint(0, W - w - 1)
    x2 = x1 + w
    y1 = random.randint(0, H - h - 1)
    y2 = y1 + h
    return x1, x2, y1, y2

def accuracy(input, target):
    return 100 * float(np.count_nonzero(input == target)) / target.size

def sliding_window(top, step=10, window_size=(32,32)):
    """ Slide a window_shape window across the image with a stride of step """
    for x in range(0, top.shape[0], step):
        if x + window_size[0] > top.shape[0]:
            x = top.shape[0] - window_size[0]
        for y in range(0, top.shape[1], step):
            if y + window_size[1] > top.shape[1]:
                y = top.shape[1] - window_size[1]
            yield x, y, window_size[0], window_size[1]

def count_sliding_window(top, step=10, window_size=(32,32)):
    """ Count the number of windows in an image """
    c = 0
    for x in range(0, top.shape[0], step):
        if x + window_size[0] > top.shape[0]:
            x = top.shape[0] - window_size[0]
        for y in range(0, top.shape[1], step):
            if y + window_size[1] > top.shape[1]:
                y = top.shape[1] - window_size[1]
            c += 1
    return c

def grouper(n, iterable):
    """ Browse an iterator by chunk of n elements """
    it = iter(iterable)
    while True:
        chunk = tuple(itertools.islice(it, n))
        if not chunk:
            return
        yield chunk
def swap_a(ar):
    ar1 = np.swapaxes(ar,1,3)
    ar2 = np.swapaxes(ar1,2,3)
    return ar2
def test(net, img_arr,N_CLASSES,stride=WINDOW_SIZE[0], batch_size=BATCH_SIZE, window_size=WINDOW_SIZE):
    
    # Use the network on the test set
    all_preds = []
    all_probs=[]
    all_gts = []
    net.cuda()
    net.eval()
    net.to(torch.double)
    with torch.no_grad():
        for img in img_arr:
            #print (img.shape)
            pred = np.zeros(img.shape[:2] + (N_CLASSES,))
            prob = np.zeros(img.shape[:2] + (N_CLASSES,))
            for i, coords in enumerate(grouper(batch_size, sliding_window(img, step=stride, window_size=window_size))):
                image_patches_1=[]
                image_patches = [np.copy(np.expand_dims(img[x:x+w, y:y+h],axis=0)) for x,y,w,h in coords]
                image_patches_1= np.concatenate(image_patches, axis=0)
                #print()
                
                #swap axes
                image_patches_1= swap_a(image_patches_1)
                print("images_patches_1.shape",image_patches_1.shape)

                image_patches_1 = Variable(torch.from_numpy(image_patches_1).cuda())
                # Do the inference
                #outs = net.classifier(net.feature(image_patches_1))
                outs,dom_out = net(x=image_patches_1)
                #sgm = torch.nn.Sigmoid()
                #outs = sgm(outs)
                del image_patches # clear mem

                outs = outs.data.cpu().numpy()
                print(outs.shape)
                # Fill in the results array
                for out, (x, y, w, h) in zip(outs, coords):
                    out = out.transpose((1,2,0))
                    #pred[x:x+w, y:y+h] += out  #this creates addition in the overlap regions
                    print("out shape",out.shape)

                    pred[x:x+w, y:y+h] = out
                del(outs)
            "use the np max for max probability"
            #prob = np.max(pred, axis=2)
            pred = np.argmax(pred, axis=2)
            #print("pred shape",pred.shape)

            all_preds.append(pred)
            all_probs.append(prob)

    return all_preds,all_probs

#Reclassifying the array and giving it coordinates
def reclass_gts(gtsarray):
    """Reclassify ground truth dataset array to single class numbers.

    Keyword arguments:
    gtsarray -- the ground truth dataset array
    """
    reclassarray = np.zeros(shape=(gtsarray.shape[0], gtsarray.shape[1]),
                            dtype=np.uint8)
    cnum = 1
    for color in _ccolors:
        mask = np.logical_and(np.logical_and(gtsarray[:, :, 0] == color[0],
                              gtsarray[:, :, 1] == color[1]),
                              gtsarray[:, :, 2] == color[2])
        reclassarray[mask] = cnum
        cnum += 1
    return reclassarray

def save_to_1band(arr,out_path,geom,proj):
    #save a one band image
    #p_rec is the reclassified array and has two dimensions
    #pred_map_GCs2 = predsdir + "trainingset_3/georeferenced/fcn_atr_goma_1947_clip1_1bands.tif"
    arr=np.expand_dims(arr, axis=2)
    nrows,ncols,nbands = arr.shape[0],arr.shape[1],arr.shape[2]
    driver = gdal.GetDriverByName('GTiff')
    nw_ds = driver.Create(out_path, ncols, nrows, nbands, gdal.GDT_UInt32 )
    nw_ds.SetGeoTransform(geom)
    nw_ds.SetProjection(proj)

    nw_ds.GetRasterBand(1).WriteArray(arr[:, :,0])

    #for i in range(nbands):
    #	nw_ds.GetRasterBand(i+1).WriteArray(P_rec[:, :, i])

    nw_ds = None

def save_prob_1band(arr,out_path,geom,proj):
    #save a one band image
    #p_rec is the reclassified array and has two dimensions
    #pred_map_GCs2 = predsdir + "trainingset_3/georeferenced/fcn_atr_goma_1947_clip1_1bands.tif"
    arr=np.expand_dims(arr, axis=2)
    nrows,ncols,nbands = arr.shape[0],arr.shape[1],arr.shape[2]
    driver = gdal.GetDriverByName('GTiff')
    nw_ds = driver.Create(out_path, ncols, nrows, nbands, gdal.GDT_Float32 )
    nw_ds.SetGeoTransform(geom)
    nw_ds.SetProjection(proj)

    nw_ds.GetRasterBand(1).WriteArray(arr[:, :,0])

    #for i in range(nbands):
    #	nw_ds.GetRasterBand(i+1).WriteArray(P_rec[:, :, i])

    nw_ds = None

# normalize the top array
def norm_rgbn(data,gmax,gmin):
    """
    rexcale  the multispectral data [0,1]
    data--the multispectral array
    """
    data = data.astype(float)
    data_norm = (data - gmin)/(gmax - gmin)
    return data_norm

def data_proj(arr_path):
    raster_dataset = gdal.Open(arr_path, gdal.GA_ReadOnly)
    geo_transform = raster_dataset.GetGeoTransform()
    proj = raster_dataset.GetProjectionRef()
    return geo_transform,proj


# ACCURACY METRICS FUNCTIONS 
def create_mask_from_vector(vector_data_path, cols, rows, geo_transform,
                            projection, target_value=1):
    """Rasterize the given vector (wrapper for gdal.RasterizeLayer)."""
    data_source = gdal.OpenEx(vector_data_path, gdal.OF_VECTOR)
    layer = data_source.GetLayer(0)
    driver = gdal.GetDriverByName('MEM')  # In memory dataset
    target_ds = driver.Create('', cols, rows, 1, gdal.GDT_UInt16)
    target_ds.SetGeoTransform(geo_transform)
    target_ds.SetProjection(projection)
    #gdal.RasterizeLayer(target_ds, [1], layer, burn_values=[target_value])
    gdal.RasterizeLayer(target_ds, [1], layer, options =["ATTRIBUTE=class_id"])    
    return target_ds
    #gdal.RasterizeLayer()

def vectors_to_raster(path, rows, cols, geo_transform, projection):
    """Rasterize the vectors in the given directory in a single image."""
    labeled_pixels = np.zeros((rows, cols))
    label = 1
    ds = create_mask_from_vector(path, cols, rows, geo_transform,
                                 projection, target_value=label)
    band = ds.GetRasterBand(1)
    labeled_pixels += band.ReadAsArray()
    ds = None
    return labeled_pixels

def write_geotiff(fname, data, geo_transform, projection):
    """Create a GeoTIFF file with the given data."""
    driver = gdal.GetDriverByName('GTiff')
    rows, cols = data.shape
    dataset = driver.Create(fname, cols, rows, 1, gdal.GDT_Byte)
    dataset.SetGeoTransform(geo_transform)
    dataset.SetProjection(projection)
    band = dataset.GetRasterBand(1)
    band.WriteArray(data)
    dataset = None  # Close the file
    
# convert the geotiff to a numpy array

def gtiff_to_array(imgfname):
    """Transform a geotiff to numpy array.

    Keyword arguments:
    imgfnames -- filename of image to convert
    """
    ds = gdal.Open(imgfname)
    for band in range(ds.RasterCount):
        band += 1
        if band == 1:
            arr = np.array(ds.GetRasterBand(band).ReadAsArray())
            arr = np.expand_dims(arr, axis=2)
        else:
            concat = np.array(ds.GetRasterBand(band).ReadAsArray())
            concat = np.expand_dims(concat, axis=2)
            arr = np.concatenate((arr,
                                  concat),
                                 axis=2)
    return arr

def img_to_array(*images):
    """Convert an image or list of images to numpy arrays.

    Keyword arguments:
    *images -- list containing the images to be converted
    """
    imgarrays = []
    i = 0
    for img in images:
        arr = gtiff_to_array(img)
        imgarrays.append(arr)
    return imgarrays

# sampling an array
def sample_idx(arr):
    """Randomly sample an array
    """
    i = 0
    arr_copy = np.copy(arr)
    idxarray = np.zeros(shape=(0, 2), dtype=np.int16)
    nc = np.array(np.where(arr_copy >0)).T.shape[0]
    #find how to automatically select the samples
    randidx = np.random.choice(range(nc),
                               size=nc, replace=False)
    randidx=randidx.astype(np.int32)
    idxarray = np.array(np.where(arr_copy > 0)).T[randidx, :]

    del arr_copy
    return idxarray

def extract_values(f,idx):
    #extract values from an array 
    samples=[]
    samples.append(np.asarray([f[rc[0],rc[1], ]
                                          for rc in idx]))
    samples_arr=np.concatenate(samples, axis=0)
    return samples_arr
    

def sample_idx_freq(arr,cratios,n):
    """Randomly stratified array sampling
    """
    arr_copy = np.copy(arr)
    idxarray = np.zeros(shape=(0, 2), dtype = np.int16)
    nclasses = len(cratios)

    for i in range (nclasses):
        csamples = int(round(n*cratios[i]))
        nc = np.array(np.where(arr_copy==i+1)).T.shape[0]
        print(nc)
        if nc>0:
            randidx = np.random.choice(range(nc),
                               size = csamples, replace = True)
            idxarray = np.concatenate((idxarray,
                                   np.array(np.where(arr_copy == i+1)).T
                                   [randidx, :]),
                                  axis=0)
    del arr_copy
    return idxarray
#use a fixed test set
def extract_values(f,idx):
    #extract values from an array 
    samples=[]
    samples.append(np.asarray([f[rc[0],rc[1], ]
                                          for rc in idx]))
    samples_arr=np.concatenate(samples, axis=0)
    return samples_arr

def make_dir_paths(mypath):
    if not os.path.isdir(mypath):
        os.makedirs(mypath)
        print("created path")
        
def reclass_preds(arr, value, reclass_value):
    
    arr_copy=np.copy(arr)
    
    for i in range(len(arr_copy)):
        if arr_copy[i]>value:
            arr_copy[i]=reclass_value
    
    return arr_copy


#SLICE_1 = [100,200,400,800,1600,2000]
#SLICE_2 = [25,50,100,200,400,500]
#SLICE_1 = [400]
#SLICE_2 = [100]
#SLICE_1 = [100,200,400]
#SLICE_2 = [25,50,100]
#SLICE_1 = [100,200,300]
#SLICE_2 = [25,50,75]
#SLICE_1 = [50,400,800]
#SLICE_2 = [25,50,100]

SLICE_3 = [300,450,600] # numbr of samples to draw
seed_value = [10,100,500,1000,10000]


src_data_dir ='H:/DA_ETH/DRC_BALANCED/buk_59.hdf5'
tgt_data_dir ='H:/DA_ETH/DRC_BALANCED/gom_47.hdf5'

for j in seed_value:
    
    for i in range(len(SLICE_1)):
        #JOB_ID = str(SLICE_1[i]+2*SLICE_2[i])
        JOB_ID = str(SLICE_3[i])+"_"+str(j)
        w8_fname = 'H:/DA_ETH/DRC_BALANCED/WEIGHTS/'+JOB_ID+'_FT_CORAL_gom_src_150_stat_pp4.hdf5'
        model = UNET().to(device)

        w8_ft_fname = "H:/DA_ETH/DRC_BALANCED/WEIGHTS/150_CORAL_goma_src.hdf5"
        model.load_state_dict(torch.load(w8_ft_fname))

        base_lr = 0.001
        #optimizer = optim.SGD(model.parameters(), lr=base_lr, momentum=0.9, weight_decay=0.0005)
        optimizer = optim.Adam(model.parameters(), lr=base_lr)
        src_dataset = MyDataset_BK(src_data_dir,slice_3=SLICE_3[i],seed_value = j)
        tgt_dataset = MyDataset_GM(tgt_data_dir,slice_3=SLICE_3[i],seed_value = j)
        #using the data loader
        src_dataloader = data.DataLoader(dataset = src_dataset, batch_size = 8, shuffle = True)
        tgt_dataloader = data.DataLoader(dataset = tgt_dataset, batch_size = 8, shuffle = True)

        t0 = time.time()
        loss_history = train(model,optimizer,50,src_dataloader,tgt_dataloader,device, scheduler=None)
        t1 = time.time()

        print("training complete in %0.2f"%((t1-t0)/60))
        #val_loss_history, tra_acc_history, coral_loss_history, src_cl_loss_history
        # save the loss values
        loss_folder = 'H:/DA_ETH/DRC_BALANCED/FIGURES/'+JOB_ID+'_FT_CORAL_loss_gom_src_150_stat_pp4.hdf5'
        with h5.File(loss_folder,"w") as f:
            f['loss_tot'] = loss_history[0]
            f['tra_acc'] = loss_history[1]
            f['dom_acc'] = loss_history[2]
            f['src_cl_loss'] = loss_history[3]
            #f['src_dl_loss'] = loss_history[4]
            #f['tgt_dl_loss'] = loss_history[5]

        # testing part fits in here

        EXPT_ID = 'GM_S_CORAL_FT_150_stat_PP4_'+JOB_ID
        root_path = "H:/DRC_THESIS/DNN_LABELLED_DATA/buk_prel_test_25_11"

        raw_files_pan = glob.glob(root_path+'/raw_tif_totest/*.tif')
        mini_preds_folder = root_path+"/DOMAIN_ADAPT/RESULTS/"+EXPT_ID+"/predictions/MINI_TILES"
        probs_folder = root_path+"/DOMAIN_ADAPT/RESULTS/"+EXPT_ID+"/probabilities/WHOLE_TILE_PROBS"

        make_dir_paths(probs_folder)
        make_dir_paths(mini_preds_folder)


        ## TEST MODEL
        t0 = time.time()

        dom_net= UNET()
        dom_net.load_state_dict(torch.load(w8_fname))
        #dom_net = dom_net.classifier

        for rgbn in raw_files_pan[:]:
            rgbn_raw=img_to_array(rgbn)
            geo_info,projec_info=data_proj(rgbn)

            #normalize the rgb
            rgbn_norm=norm_rgbn(rgbn_raw[0],gmax,gmin)
            del rgbn_raw
            #rgbn_norm = swap_a(rgbn_norm)

            allpred= test(dom_net,[rgbn_norm],nc,stride=STRIDE)

            file_name=os.path.split(rgbn)[-1]
            save_to_1band(allpred[0][0]+1,mini_preds_folder+"/"+file_name,geo_info,projec_info)
            #save_prob_1band(allpred[0][0],mini_preds_folder+"/"+file_name,geo_info,projec_info)
            #save_to_1band(allpred[0][0]+1,mini_preds_folder+"/"+file_name,geo_info,projec_info)
            #save_prob_1band(allpred[1][0],probs_folder+"/"+file_name,geo_info,projec_info)

        t1 = time.time()
        print("Finished testing after %.2f mins" % ((t1-t0)/60.0))


        #COMPUTE THE ACCURACY METRICS

        txtcontent = ""

        logs_folder=root_path+"/RESULTS_METRICS_6CLASSES/"+EXPT_ID+"/FIGURES_TIME_LOGS"
        make_dir_paths(logs_folder)

        class_path = root_path+"/DOMAIN_ADAPT/RESULTS/"+EXPT_ID+"/predictions/MINI_TILES"
        #gts_files = glob.glob(root_path+"/gts_tif_totest/*.tif")
        gts_files = glob.glob(root_path+"/gts_tif2_test/*.tif")
        class_files= glob.glob(class_path+'/*.tif')

        t0 = time.time()
        num_tiles=len(gts_files)
        counter = 0
        all_gts=[]
        all_labeled=[]

        for gts,classified in zip(gts_files,class_files):
            counter += 1
            print("Tile %i of %i" % (counter, num_tiles))
            #importing the rasters
            refs,labeled= img_to_array(gts,classified)

            #reshape reference
            refs = np.reshape (refs,(refs.shape[0],refs.shape[1]))
            #reshape labels
            labeled = np.reshape(labeled,(labeled.shape[0],labeled.shape[1]))

            #idxarray = sample_idx_freq(refs,cratios,n) #
            idxarray = sample_idx(refs) #

            #extract the gts pixels into an array
            idx_gts= extract_values(refs,idxarray)

            #extract the pred pixels into an array
            idx_pred=extract_values(labeled,idxarray)

            #trainset=trainset[rand_list]
            all_gts.append(idx_gts)
            all_labeled.append(idx_pred)

        #saving    
        all_gts_arr=np.asarray(all_gts)
        gts_set= np.concatenate(all_gts, axis=0)

        all_preds_arr=np.asarray(all_labeled)
        pred_set= np.concatenate(all_preds_arr, axis=0)

        LABELS = ["1","2","3","4","5","6"] 

        """
        ####RECLASSIFYING TO THREE CLASSES ## if you want to reclassify the Mixed bare class/ Low vegetation sub-classes 
        gts_set=reclass_preds(gts_set,3,3)
        pred_set=reclass_preds(pred_set,3,3)
        LABELS = ["1","2","3"] 
        """

        ####CONFUSION MATRIX
        #cm=confusion_matrix(gts_set, pred_set)

        cm=confusion_matrix(gts_set, pred_set)

        messagetoprint = "Confusion matrix:\n%s" %cm
        txtcontent+=messagetoprint+"\n\n"

        #OVERALL ACCURACY

        messagetoprint="overall accuracy: %f" %metrics.accuracy_score(gts_set, pred_set)
        txtcontent+=messagetoprint+"\n\n"

        ##TOTAL NUMBER OF PIXELS PROCESSED

        total = sum(sum(cm))
        messagetoprint = "total number of pixels processed:\n%s" %total
        txtcontent+=messagetoprint+"\n\n"

        ###PRECISION AND RECALL
        messagetoprint="Classification report:\n\n%s" %metrics.classification_report(gts_set, 
                                            pred_set,
                                            target_names=LABELS)
        #print (messagetoprint)
        txtcontent+=messagetoprint+"\n\n"

        ####PRODUCER AND USER ACCURACY

        PA = [cm[x][x]/np.sum(cm[x,:]) for x in range(len(cm))]

        UA = [cm[x][x]/np.sum(cm[:,x]) for x in range(len(cm))]

        PA_rounded = list(np.around(np.array(PA),4))
        PA_dict={k: v for k, v in zip(LABELS, PA_rounded)}
        messagetoprint="PRODUCER ACCURACY:\n%s" %PA_dict
        txtcontent+=messagetoprint+"\n"

        UA_rounded=list(np.around(np.array(UA),4))
        UA_dict={k: v for k, v in zip(LABELS, UA_rounded)}
        messagetoprint="USER ACCURACY:\n%s" %UA_dict
        txtcontent+=messagetoprint+"\n"

        f = open(logs_folder+"/accuracy_metrics.txt", 'w')
        f.write(EXPT_ID+" metrics"+"\n\n")
        f.write(txtcontent)
        f.close()


        # FOR THE SECOND TILE

        root_path = "H:/DRC_THESIS/DNN_LABELLED_DATA/goma_prel_test_25_11"

        raw_files_pan = glob.glob(root_path+'/raw_tif_totest/*.tif')
        mini_preds_folder = root_path+"/DOMAIN_ADAPT/RESULTS/"+EXPT_ID+"/predictions/MINI_TILES"
        probs_folder = root_path+"/DOMAIN_ADAPT/RESULTS/"+EXPT_ID+"/probabilities/WHOLE_TILE_PROBS"

        make_dir_paths(probs_folder)
        make_dir_paths(mini_preds_folder)


        ## TEST MODEL
        t0 = time.time()

        dom_net= UNET()
        dom_net.load_state_dict(torch.load(w8_fname))
        #dom_net = dom_net.classifier

        for rgbn in raw_files_pan[:]:
            rgbn_raw=img_to_array(rgbn)
            geo_info,projec_info=data_proj(rgbn)

            #normalize the rgb
            rgbn_norm=norm_rgbn(rgbn_raw[0],gmax,gmin)
            del rgbn_raw
            #rgbn_norm = swap_a(rgbn_norm)

            allpred= test(dom_net,[rgbn_norm],nc,stride=STRIDE)

            file_name=os.path.split(rgbn)[-1]
            save_to_1band(allpred[0][0]+1,mini_preds_folder+"/"+file_name,geo_info,projec_info)
            #save_prob_1band(allpred[0][0],mini_preds_folder+"/"+file_name,geo_info,projec_info)
            #save_to_1band(allpred[0][0]+1,mini_preds_folder+"/"+file_name,geo_info,projec_info)
            #save_prob_1band(allpred[1][0],probs_folder+"/"+file_name,geo_info,projec_info)

        t1 = time.time()
        print("Finished testing after %.2f mins" % ((t1-t0)/60.0))


        #COMPUTE THE ACCURACY METRICS

        txtcontent = ""

        logs_folder=root_path+"/RESULTS_METRICS_6CLASSES/"+EXPT_ID+"/FIGURES_TIME_LOGS"
        make_dir_paths(logs_folder)

        class_path = root_path+"/DOMAIN_ADAPT/RESULTS/"+EXPT_ID+"/predictions/MINI_TILES"
        #gts_files = glob.glob(root_path+"/gts_tif_totest/*.tif")
        gts_files = glob.glob(root_path+"/gts_tif2_test/*.tif")
        class_files= glob.glob(class_path+'/*.tif')

        t0 = time.time()
        num_tiles=len(gts_files)
        counter = 0
        all_gts=[]
        all_labeled=[]

        for gts,classified in zip(gts_files,class_files):
            counter += 1
            print("Tile %i of %i" % (counter, num_tiles))
            #importing the rasters
            refs,labeled= img_to_array(gts,classified)

            #reshape reference
            refs = np.reshape (refs,(refs.shape[0],refs.shape[1]))
            #reshape labels
            labeled = np.reshape(labeled,(labeled.shape[0],labeled.shape[1]))

            #idxarray = sample_idx_freq(refs,cratios,n) #
            idxarray = sample_idx(refs) #

            #extract the gts pixels into an array
            idx_gts= extract_values(refs,idxarray)

            #extract the pred pixels into an array
            idx_pred=extract_values(labeled,idxarray)

            #trainset=trainset[rand_list]
            all_gts.append(idx_gts)
            all_labeled.append(idx_pred)

        #saving    
        all_gts_arr=np.asarray(all_gts)
        gts_set= np.concatenate(all_gts, axis=0)

        all_preds_arr=np.asarray(all_labeled)
        pred_set= np.concatenate(all_preds_arr, axis=0)

        LABELS = ["1","2","3","4","5","6"] 

        """
        ####RECLASSIFYING TO THREE CLASSES ## if you want to reclassify the Mixed bare class/ Low vegetation sub-classes 
        gts_set=reclass_preds(gts_set,3,3)
        pred_set=reclass_preds(pred_set,3,3)
        LABELS = ["1","2","3"] 
        """

        ####CONFUSION MATRIX
        #cm=confusion_matrix(gts_set, pred_set)

        cm=confusion_matrix(gts_set, pred_set)

        messagetoprint = "Confusion matrix:\n%s" %cm
        txtcontent+=messagetoprint+"\n\n"

        #OVERALL ACCURACY

        messagetoprint="overall accuracy: %f" %metrics.accuracy_score(gts_set, pred_set)
        txtcontent+=messagetoprint+"\n\n"

        ##TOTAL NUMBER OF PIXELS PROCESSED

        total = sum(sum(cm))
        messagetoprint = "total number of pixels processed:\n%s" %total
        txtcontent+=messagetoprint+"\n\n"

        ###PRECISION AND RECALL
        messagetoprint="Classification report:\n\n%s" %metrics.classification_report(gts_set, 
                                            pred_set,
                                            target_names=LABELS)
        #print (messagetoprint)
        txtcontent+=messagetoprint+"\n\n"

        ####PRODUCER AND USER ACCURACY

        PA = [cm[x][x]/np.sum(cm[x,:]) for x in range(len(cm))]

        UA = [cm[x][x]/np.sum(cm[:,x]) for x in range(len(cm))]

        PA_rounded = list(np.around(np.array(PA),4))
        PA_dict={k: v for k, v in zip(LABELS, PA_rounded)}
        messagetoprint="PRODUCER ACCURACY:\n%s" %PA_dict
        txtcontent+=messagetoprint+"\n"

        UA_rounded=list(np.around(np.array(UA),4))
        UA_dict={k: v for k, v in zip(LABELS, UA_rounded)}
        messagetoprint="USER ACCURACY:\n%s" %UA_dict
        txtcontent+=messagetoprint+"\n"

        f = open(logs_folder+"/accuracy_metrics.txt", 'w')
        f.write(EXPT_ID+" metrics"+"\n\n")
        f.write(txtcontent)
        f.close()



(array([-1.,  0.,  1.,  2.,  3.,  4.,  5.]), array([274278,  65893, 212318, 284540, 522779, 237628,  40964],
      dtype=int64))
(array([-1.,  0.,  1.,  2.,  3.,  4.,  5.]), array([155925,  36584, 308603, 322332, 378916, 321349, 114691],
      dtype=int64))
epoch number 1
----------
epoch number 2
----------
epoch number 3
----------
epoch number 4
----------
epoch number 5
----------
epoch number 6
----------
epoch number 7
----------
epoch number 8
----------
epoch number 9
----------
epoch number 10
----------
epoch number 11
----------
epoch number 12
----------
epoch number 13
----------
epoch number 14
----------
epoch number 15
----------
epoch number 16
----------
epoch number 17
----------
epoch number 18
----------
epoch number 19
----------
epoch number 20
----------
epoch number 21
----------
epoch number 22
----------
epoch number 23
----------
epoch number 24
----------
epoch number 25
----------
epoch number 26
----------
epoch number 27
----------
epoch number 28
------